In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


In [2]:
#COMANDO PARA NÃO QUEBRAR LINHA NO DATAFRAME
pd.set_option('expand_frame_repr', False)

In [3]:
#PARAMETROS PARA CONSULTA SQL
hostname = 'localhost'
database = 'dev_lucas'
username = 'postgres'
pwd = 'admin'

engine = create_engine('postgresql+psycopg2://' + username + ':' + pwd + '@' + hostname + '/' + database)

with engine.connect() as conn:
    sql = 'SELECT * from ifdata_pj'
    df2 = pd.read_sql(sql, con = conn)


In [4]:
#QUESTOES
#Q1 - MAIORES INSTITUIÇÕES - PJ
#Q2 - PRINCIPAIS PRODUTOS / INAD - PJ
#Q4 - PRINCIPAL PRODUTO

In [5]:
Q1 = df2.assign(
    vencido = np.where(df2['nomecoluna'].str.contains('VENCIDO'),df2['saldo'],0),
    vencer = np.where(df2['nomecoluna'].str.contains('VENCER'),df2['saldo'],0),
    total = df2['saldo']
    ).groupby('codinst').agg({'vencido':sum, 'vencer':sum, 'total':sum}).sort_values(by='total', ascending=False)

inad = Q1['vencido'] / Q1['total']  
Q1 = Q1.assign(inad=inad)

# TRATANDO VALORES NÃO NUMÉRICOS PARA NaN
Q1['inad'] = pd.to_numeric(Q1['inad'], errors='coerce')

# PREENCHENDO NaN COM 0 E FORMATANDO ESTES E DEMAIS COM PERCENTUAL
Q1['inad'] = Q1['inad'].fillna(0).apply(lambda x: '{:.2%}'.format(x))

# TRATANDO VALORES NUMÉRICOS PARA BILHÕES OU MILHÕES
def convert_to_billion(x):
    if x > 1000000000:
        return str(round((x/1000000000),2)) + 'B'
    else: 
        return str(round((x/1000000),2)) + 'M'

Q1['vencido'] = Q1['vencido'].apply(lambda x : convert_to_billion(x))
Q1['vencer'] = Q1['vencer'].apply(lambda x : convert_to_billion(x))
Q1['total'] = Q1['total'].apply(lambda x : convert_to_billion(x))


print(Q1)

          vencido   vencer    total   inad
codinst                                   
C0049906    3.18B  279.91B  283.09B  1.12%
33657248  194.87M  278.09B  278.28B  0.07%
C0010045    4.39B  252.56B  256.94B  1.71%
C0010069    2.87B  224.33B   227.2B  1.26%
C0051626    2.02B  210.47B  212.49B  0.95%
...           ...      ...      ...    ...
48584954     0.0M     0.0M     0.0M  0.00%
48176336     0.0M     0.0M     0.0M  0.00%
48172860     0.0M     0.0M     0.0M  0.00%
48140925     0.0M     0.0M     0.0M  0.00%
23241540     0.0M     0.0M     0.0M  0.00%

[1358 rows x 4 columns]


In [9]:
Q2 = df2.assign(
    vencido = np.where(df2['nomecoluna'].str.contains('VENCIDO'),df2['saldo'],0),
    vencer = np.where(df2['nomecoluna'].str.contains('VENCER'),df2['saldo'],0),
    total = df2['saldo']
    ).groupby('grupo', as_index=False).agg({'vencido':sum, 'vencer':sum, 'total':sum}).sort_values(by='total', ascending=False)

inad = Q2['vencido'] / Q2['total']  
Q2 = Q2.assign(inad=inad)

# TRATANDO VALORES NÃO NUMÉRICOS PARA NaN
Q2['inad'] = pd.to_numeric(Q2['inad'], errors='coerce')

# PREENCHENDO NaN COM 0 E FORMATANDO ESTES E DEMAIS COM PERCENTUAL
Q2['inad'] = Q2['inad'].fillna(0).apply(lambda x: '{:.2%}'.format(x))

# TRATANDO VALORES NUMÉRICOS PARA BILHÕES OU MILHÕES
# COMO A FUNÇÃO FOI DEFINIDA NA ETAPA ANTERIOR, NÃO É NECESSARIO REFAZE-LA
Q2['vencido'] = Q2['vencido'].apply(lambda x : convert_to_billion(x))
Q2['vencer'] = Q2['vencer'].apply(lambda x : convert_to_billion(x))
Q2['total'] = Q2['total'].apply(lambda x : convert_to_billion(x))

print(Q2)

                                               grupo  vencido   vencer    total   inad
0                                    Capital de Giro     6.6B  593.18B  599.78B  1.10%
3  Financiamento de Infraestrutura/Desenvolviment...  985.91M  505.38B  506.36B  0.19%
6                           Operações com Recebíveis    1.77B  247.17B  248.94B  0.71%
5                                       Investimento    1.09B  247.57B  248.66B  0.44%
2                                  Comércio Exterior   710.4M  227.32B  228.03B  0.31%
7                                    Outros Créditos    6.06B   132.0B  138.05B  4.39%
1                  Cheque Especial e Conta Garantida    3.53B    47.5B   51.03B  6.92%
8                             Rural e Agroindustrial    87.9M   47.74B   47.83B  0.18%
4                                       Habitacional  164.32M    25.4B   25.57B  0.64%


In [10]:
#QUESTOES - PJ
#Q1 - MAIORES INSTITUIÇÕES - PJ
#Q2 - PRINCIPAIS PRODUTOS / INAD - PJ
#Q4 - PRINCIPAL PRODUTO
print('As 5 maiores instituições, em tamanho da carteira de crédito para pessoa jurídica, são: \n {} \n '.format(Q1.head(5)))
print('Os 5 principais produtos da carteira de crédito para pessoa jurídica e suas respectivas taxas de inadimplência, são: \n {} \n '.format(Q2.head(5)))
print('O produto principal dentro da carteira de crédito para pessoa jurídica é o {} que possui uma carteira de {} e uma taxa de inadimplência \
      de {}.'.format(Q2['grupo'].iloc[0], Q2['total'].iloc[0], Q2['inad'].iloc[0]))

As 5 maiores instituições, em tamanho da carteira de crédito para pessoa jurídica, são: 
           vencido   vencer    total   inad
codinst                                   
C0049906    3.18B  279.91B  283.09B  1.12%
33657248  194.87M  278.09B  278.28B  0.07%
C0010045    4.39B  252.56B  256.94B  1.71%
C0010069    2.87B  224.33B   227.2B  1.26%
C0051626    2.02B  210.47B  212.49B  0.95% 
 
Os 5 principais produtos da carteira de crédito para pessoa jurídica e suas respectivas taxas de inadimplência, são: 
                                                grupo  vencido   vencer    total   inad
0                                    Capital de Giro     6.6B  593.18B  599.78B  1.10%
3  Financiamento de Infraestrutura/Desenvolviment...  985.91M  505.38B  506.36B  0.19%
6                           Operações com Recebíveis    1.77B  247.17B  248.94B  0.71%
5                                       Investimento    1.09B  247.57B  248.66B  0.44%
2                                  Comércio Exterior